In [ ]:
# default_exp clone_analysis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import reduce

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Clone Analysis Functions

## data wrangling functions

In [ ]:
# export
def _combine_agg_functions(additional_agg_functions):
    if additional_agg_functions is None:
        additional_agg_functions = {}

    agg_functions = {"label": "count", "area_um2": [np.mean, np.std]}
    return {**agg_functions, **additional_agg_functions}

In [ ]:
# export
def _individual_filter_condition(
    df, filtered_col_name: str, query: str, clone_channel: str, agg_functions
):
    if query is not None:
        df = df.query(query)

    temp_df = (df.groupby(["int_img", clone_channel]).agg(agg_functions)).copy()

    temp_df.columns = pd.MultiIndex.from_tuples(
        [(filtered_col_name,) + a for a in temp_df.columns]
    )
    return temp_df

In [ ]:
# export
def query_df_groupby_by_clone_channel(
    df, queries: dict, clone_channel: str = "C1", additional_agg_functions: dict = None,
):
    """additional agg_functions could be something like:
    additional_agg_functions = {"mean_intensity": [np.mean, np.std]}"""

    agg_functions = _combine_agg_functions(additional_agg_functions)
    df = df.reset_index()

    l = list()
    for key, query in queries.items():
        l.append(
            _individual_filter_condition(df, key, query, clone_channel, agg_functions)
        )

    return reduce(
        lambda df_left, df_right: pd.merge(
            df_left, df_right, how="outer", left_index=True, right_index=True
        ),
        l,
    )

## data visualization functions

In [ ]:
def create_stack_bar_plot(
    df,
    df_error_bar=None,
    x_figSize=2.5,
    y_figSize=2.5,
    y_label=None,
    y_axis_start=0,
    y_axis_limit=None,
    color_pal=sns.color_palette(palette="Blues_r"),
    bar_width=0.8,
):

    fig, ax = plt.subplots(figsize=(x_figSize, y_figSize))

    sns.set(style="ticks")
    sns.despine()

    ax = DF.plot(
        kind="bar",
        stacked=True,
        color=color_pal,
        width=bar_width,
        ax=ax,
        yerr=DF_error_bar,
        capsize=4,
    )
    ax.set_ylabel(y_label, fontsize=12)
    sns.despine(ax=ax)
    ax.xaxis.set_tick_params(width=1)
    ax.yaxis.set_tick_params(width=1)
    ax.tick_params(axis="both", which="major", pad=1)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.setp(ax.spines.values(), linewidth=1)

    if not y_axis_limit == None:
        ax.set_ylim(top=y_axis_limit)

    handles, labels = ax.get_legend_handles_labels()

    ax.legend(
        reversed(handles), reversed(labels), bbox_to_anchor=(1, 1), loc="upper left"
    )

In [ ]:
from dask.distributed import Client

from py_clone_detective.clone_counters import LazyCloneCounter

In [ ]:
c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56460,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:56469,Total threads: 2
Dashboard: http://127.0.0.1:56471/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:56463,


## Example using LazyCloneCounter with measure_overlap

In [ ]:
bar = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

bar.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

bar.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
bar.make_measurements(extra_properties=["convex_area"],)
bar.measure_overlap()
bar.combine_C0_overlaps_and_measurements()

## Example using LazyCloneCounter with add_clones_and_neighbouring_labels

In [ ]:
from skimage import morphology

In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

In [ ]:
foo.add_segmentations(
    morphology.remove_small_objects,
    ad_func_kwargs={"min_size": 49},
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
)

In [ ]:
foo.make_measurements()

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C1" & mean_intensity > 1000',
    name_for_query="C1",
    calc_clones=True,
)

In [ ]:
foo.results_measurements = foo.results_measurements.eval(
    "total_intensity = mean_intensity * area"
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C2" & total_intensity > 5e5',
    name_for_query="C2",
    calc_clones=False,
)

In [ ]:
foo.add_clones_and_neighbouring_labels(
    query_for_pd='int_img_ch == "C3" & mean_intensity > 1000',
    name_for_query="C3",
    calc_clones=False,
)

In [ ]:
foo.measure_clones_and_neighbouring_labels(name_for_query="C1")
foo.measure_clones_and_neighbouring_labels(name_for_query="C2")
foo.measure_clones_and_neighbouring_labels(name_for_query="C3")

df = foo.combine_neighbour_counts_and_measurements()

In [ ]:
df

C1_clone  total_neighbour_counts  C1pos_nc  C1neg_nc  C2pos_nc  \
int_img label                                                                   
a1g01p1 12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        12            0                       3         1         2         3   
        13            0                       3         0         3         2   
...                 ...                     ...       ...       ...       ...   
a2g13p3 232           0                       3         0         3         1   
        233           0                       2         0         2         1   
        233           0                       2         0         2         1   
        233           0                       2         0         2         1   
        233           0                       2         0         2         1   

               C2neg_nc  C3pos_nc  C3neg_nc seg_ch int_img_ch  seg_img  area  \
int_img label                                                                  
a1g01p1 12            0         0         3     C0         C0  a1g01p1  1094   
        12            0         0         3     C0         C1  a1g01p1  1094   
        12            0         0         3     C0         C2  a1g01p1  1094   
        12            0         0         3     C0         C3  a1g01p1  1094   
        13            1         0         3     C0         C0  a1g01p1  1123   
...                 ...       ...       ...    ...        ...      ...   ...   
a2g13p3 232           2         0         3     C0         C3  a2g13p3   723   
        233           1         0         2     C0         C0  a2g13p3   253   
        233           1         0         2     C0         C1  a2g13p3   253   
        233           1         0         2     C0         C2  a2g13p3   253   
        233           1         0         2     C0         C3  a2g13p3   253   

               mean_intensity  centroid-0  centroid-1   area_um2  C1_pos  \
int_img label                                                              
a1g01p1 12        2103.588665   43.901280   89.606947  82.733750   False   
        12          60.372029   43.901280   89.606947  82.733750   False   
        12        2209.449726   43.901280   89.606947  82.733750   False   
        12          73.462523   43.901280   89.606947  82.733750   False   
        13        3414.680321   51.479964  362.270703  84.926875   False   
...                       ...         ...         ...        ...     ...   
a2g13p3 232        107.864454  772.175657  475.138313  54.676875   False   
        233       1257.762846  771.916996  675.988142  19.133125   False   
        233         32.418972  771.916996  675.988142  19.133125   False   
        233        742.395257  771.916996  675.988142  19.133125   False   
        233         99.430830  771.916996  675.988142  19.133125   False   

               total_intensity  C2_pos  C3_pos  
int_img label                                   
a1g01p1 12           2301326.0    True   False  
        12             66047.0    True   False  
        12           2417138.0    True   False  
        12             80368.0    True   False  
        13           3834686.0    True   False  
...                        ...     ...     ...  
a2g13p3 232            77986.0    True   False  
        233           318214.0   False   False  
        233             8202.0   False   False  
        233           187826.0   False   False  
        233            25156.0   False   False  

[67104 rows x 20 columns]